In [1]:
from time import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import models
from sklearn.metrics import f1_score

# … (Deine bisherigen Imports und Setup bleiben unverändert) …

# ←←← HIER BEGINNT DER BATCH-SIZE BENCHMARK CODE →→→

batch_sizes = [32, 64, 128]        # Liste der Batch-Größen, die du testen willst
results = {}

for bs in batch_sizes:
    print(f"\n=== Test mit batch_size = {bs} ===")

    # 1) Loader für Training und Validation mit neuer batch_size
    train_loader = DataLoader(
        train_dataset,               # dein bereits definierter train_dataset
        batch_size=bs,
        sampler=sampler,             # dein WeightedRandomSampler
        num_workers=2,
        pin_memory=pin_memory_flag,
        worker_init_fn=seed_worker
    )
    val_loader = DataLoader(
        val_dataset,                 # dein bereits definierter val_dataset
        batch_size=bs,
        shuffle=False,
        num_workers=2,
        pin_memory=pin_memory_flag,
        worker_init_fn=seed_worker
    )

    # 2) Modell- und Optimizer-Reset zu jedem Test
    model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.DEFAULT).to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    # 3) Kurzer Trainingslauf mit Early Stopping (5 Epochen max)
    start = time()
    best_val_loss = float('inf')
    patience      = 3
    p_cnt         = 0

    for epoch in range(1, 6):  # nur 5 Epochen zum Testen
        # Training
        model.train()
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss    = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                val_loss += criterion(model(images), labels).item() * images.size(0)
        val_loss /= len(val_loader.dataset)

        # Early Stopping Check
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            p_cnt = 0
            # optional: torch.save(model.state_dict(), 'tmp_best.pt')
        else:
            p_cnt += 1
            if p_cnt >= patience:
                print(f"→ patience {patience} erreicht, abbrechen bei epoch {epoch}")
                break

    duration = time() - start

    # 4) Finale Validation-Accuracy mit dem letzten Modellzustand
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            preds = model(images).argmax(dim=1)
            correct += (preds == labels).sum().item()
            total   += labels.size(0)
    val_acc = correct / total

    # 5) Speicherverbrauch messen (optional)
    gpu_mem = None
    if pin_memory_flag:
        gpu_mem = torch.cuda.max_memory_allocated(device) / (1024**3)

    # 6) Ergebnis speichern
    results[bs] = {
        'time_s'    : round(duration, 1),
        'val_loss'  : round(best_val_loss, 4),
        'val_acc'   : round(val_acc, 4),
        'gpu_mem_GB': round(gpu_mem, 2) if gpu_mem is not None else 'n/a'
    }

# ←←← ENDE Batch-Size Benchmark →→→

# Ausgabe der gesammelten Ergebnisse
print("\nBatch-Size Benchmark:")
for bs, r in results.items():
    print(f"bs={bs:<4}  time {r['time_s']:>5}s  loss {r['val_loss']:>6}  "
          f"acc {r['val_acc']:>5}  mem {r['gpu_mem_GB']} GB")


=== Test mit batch_size = 32 ===


NameError: name 'train_dataset' is not defined